# Part 1: Interoperability

This notebook walks through some of the interoperability aspects of data modelling when using Kùzu
with structured data. The typical workflow is to explore any existing columnar data via a DataFrame
library, or using SQL in DuckDB to study the data. It can then be modelled as a graph by seamlessly
moving between the various formats.

We will first fix issues with the column headers in the raw data and do an inspection using Pandas.

In [17]:
import polars as pl

df = pl.read_csv("winemag-data-130k-v2.csv")
df.columns

['',
 'country',
 'description',
 'designation',
 'points',
 'price',
 'province',
 'region_1',
 'region_2',
 'taster_name',
 'taster_twitter_handle',
 'title',
 'variety',
 'winery']

We can see that the first column has a blank header - this needs to be replaced so that we can use it
within DuckDB or Kùzu downstream.

In [18]:
# Rename the unnamed column to id
df = df.rename({"": "id"})
df.head()

id,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
i64,str,str,str,i64,f64,str,str,str,str,str,str,str,str
0,"""Italy""","""Aromas include tropical fruit,…","""Vulkà Bianco""",87,null,"""Sicily & Sardinia""","""Etna""",null,"""Kerin O’Keefe""","""@kerinokeefe""","""Nicosia 2013 Vulkà Bianco (Et…","""White Blend""","""Nicosia"""
1,"""Portugal""","""This is ripe and fruity, a win…","""Avidagos""",87,15.0,"""Douro""",null,null,"""Roger Voss""","""@vossroger""","""Quinta dos Avidagos 2011 Avida…","""Portuguese Red""","""Quinta dos Avidagos"""
2,"""US""","""Tart and snappy, the flavors o…",null,87,14.0,"""Oregon""","""Willamette Valley""","""Willamette Valley""","""Paul Gregutt""","""@paulgwine ""","""Rainstorm 2013 Pinot Gris (Wil…","""Pinot Gris""","""Rainstorm"""
3,"""US""","""Pineapple rind, lemon pith and…","""Reserve Late Harvest""",87,13.0,"""Michigan""","""Lake Michigan Shore""",null,"""Alexander Peartree""",null,"""St. Julian 2013 Reserve Late H…","""Riesling""","""St. Julian"""
4,"""US""","""Much like the regular bottling…","""Vintner's Reserve Wild Child B…",87,65.0,"""Oregon""","""Willamette Valley""","""Willamette Valley""","""Paul Gregutt""","""@paulgwine ""","""Sweet Cheeks 2012 Vintner's Re…","""Pinot Noir""","""Sweet Cheeks"""


We can inspect the shape of the DataFrame to see that we have scanned the full dataset. There are
14 columns, not all of which are useful for the analysis downstream.

In [19]:
df.shape

(129971, 14)

The `description` column is the one that contains the reviews for each wine. We can see some samples below.

In [20]:
df[0:5]["description"].to_list()

["Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",
 "This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.",
 'Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.',
 'Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.',
 "Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy count

## Subset the raw data
The next step is to subset the data to only include the columns that are useful for the analysis. We can
then proceed to using a persistent database, any of DuckDB, Kùzu or LanceDB to store the data.

In [21]:
# We only want to keep the following columns
wines = df.select([
    "id",
    "title",
    "country",
    "description",
    "variety",
    "points",
    "price",
    "region_1",
    "taster_name",
    "taster_twitter_handle",
])
wines.head()

id,title,country,description,variety,points,price,region_1,taster_name,taster_twitter_handle
i64,str,str,str,str,i64,f64,str,str,str
0,"""Nicosia 2013 Vulkà Bianco (Et…","""Italy""","""Aromas include tropical fruit,…","""White Blend""",87,null,"""Etna""","""Kerin O’Keefe""","""@kerinokeefe"""
1,"""Quinta dos Avidagos 2011 Avida…","""Portugal""","""This is ripe and fruity, a win…","""Portuguese Red""",87,15.0,null,"""Roger Voss""","""@vossroger"""
2,"""Rainstorm 2013 Pinot Gris (Wil…","""US""","""Tart and snappy, the flavors o…","""Pinot Gris""",87,14.0,"""Willamette Valley""","""Paul Gregutt""","""@paulgwine """
3,"""St. Julian 2013 Reserve Late H…","""US""","""Pineapple rind, lemon pith and…","""Riesling""",87,13.0,"""Lake Michigan Shore""","""Alexander Peartree""",null
4,"""Sweet Cheeks 2012 Vintner's Re…","""US""","""Much like the regular bottling…","""Pinot Noir""",87,65.0,"""Willamette Valley""","""Paul Gregutt""","""@paulgwine """


## Run SQL queries
DuckDB is an embedded relational database that makes it very simple to analyze the data using SQL.
Rather than writing out Python code in Pandas (and soon, Polars), we can directly scan the DataFrame
using SQL queries in DuckDVB. Kùzu natively scans the underlying data structures (numpy or Arrow),
making it easy to move between formats during graph construction.

The following query shows the countries with the most reviewed wines in this dataset.

In [22]:
import duckdb

# Count number of wines by country and sort in descending order
duckdb.sql(
    """
    SELECT country, COUNT(*) AS count
    FROM wines
    GROUP BY country
    ORDER BY count DESC
    """
).limit(10)

┌───────────┬───────┐
│  country  │ count │
│  varchar  │ int64 │
├───────────┼───────┤
│ US        │ 54504 │
│ France    │ 22093 │
│ Italy     │ 19540 │
│ Spain     │  6645 │
│ Portugal  │  5691 │
│ Chile     │  4472 │
│ Argentina │  3800 │
│ Austria   │  3345 │
│ Australia │  2329 │
│ Germany   │  2165 │
├───────────┴───────┤
│      10 rows      │
└───────────────────┘

Say you're on a budget and want to find wines tasted by tasters with the lowest average price.
The following query answers this.

In [23]:
# Select the wine taster with the lowest average price of wines tasted
duckdb.sql(
    """
    SELECT taster_twitter_handle, taster_name, AVG(price) AS avg_price
    FROM wines
    WHERE taster_twitter_handle IS NOT NULL
    GROUP BY taster_twitter_handle, taster_name
    ORDER BY avg_price
    """
).limit(10)

┌───────────────────────┬───────────────────┬────────────────────┐
│ taster_twitter_handle │    taster_name    │     avg_price      │
│        varchar        │      varchar      │       double       │
├───────────────────────┼───────────────────┼────────────────────┤
│ @worldwineguys        │ Jeff Jenssen      │ 22.234215885947048 │
│ @suskostrzewa         │ Susan Kostrzewa   │ 22.908667287977632 │
│ @laurbuzz             │ Lauren Buzzeo     │ 24.492702860478694 │
│ @wineschach           │ Michael Schachner │  25.23115510668183 │
│ @gordone_cellars      │ Jim Gordon        │ 26.935507072644448 │
│ @worldwineguys        │ Mike DeSimone     │  28.16468253968254 │
│ @winewchristina       │ Christina Pickard │ 29.333333333333332 │
│ @bkfiona              │ Fiona Adams       │  31.14814814814815 │
│ @AnneInVino           │ Anne Krebiehl MW  │ 31.230135373749263 │
│ @paulgwine            │ Paul Gregutt      │ 33.644872604758895 │
├───────────────────────┴───────────────────┴─────────────────

## Load the data into Kùzu

Kùzu is an embedded graph database that allows you to model your data as a graph when it makes sense
to do so. In the following example, we will model the data as a simple graph that captures the
following paths:

```
(:Taster)-[:Tastes]->(w:Wine)-[:IsFrom]->(:Country)
```

Kùzu is a disk-based GDBMS, so the data is persisted to disk and can be queried using Cypher, a
declarative query language reminiscent of SQL. Additionally, Kùzu's data model can be thought of as
a _structured_ property graph model, so we first need to instantiate a Kùzu database and
provide a schema for the graph.

In [24]:
import shutil

import kuzu

# Start with an empty database by specifying a path and a database name
shutil.rmtree("db/kuzudb", ignore_errors=True)
db = kuzu.Database("db/kuzudb")
kuzu_conn = kuzu.Connection(db)

### Schema definition
The first step is to provide node and edge tables with strong types so that Kùzu knows beforehand
what data to expect.

In [25]:
# Create wine node table
kuzu_conn.execute(
    """
    CREATE NODE TABLE 
        Wine(
            id INT64,
            title STRING,
            description STRING,
            variety STRING,
            points INT64,
            price DOUBLE,
            region_1 STRING,
            PRIMARY KEY (id)
        )
    """
)

# Create Taster node table
kuzu_conn.execute(
    """
    CREATE NODE TABLE 
        Taster(
            taster_name STRING,
            taster_twitter_handle STRING,
            PRIMARY KEY (taster_name)
        )
    """
)

# Create Country node table
kuzu_conn.execute(
    """
    CREATE NODE TABLE 
        Country(
            country STRING,
            PRIMARY KEY (country)
        )
    """
)

We also need to define relationship tables that reference the node tables' primary keys. The first
and second columns of a relationship table are always the `FROM` and `TO` primary key values.

In [26]:
# Create `Tastes` relationship table
kuzu_conn.execute("CREATE REL TABLE Tastes(FROM Taster TO Wine)")
kuzu_conn.execute("CREATE REL TABLE IsFrom(FROM Wine TO Country)")

### Scan data from desired column
The following two cells only **scan** the data from the source, they do not insert it into the graph yet.

In [27]:
kuzu_conn.execute(
    """
    LOAD FROM 'final/winemag-reviews.parquet' RETURN
        id,
        title,
        description,
        variety,
        points,
        price,
        region_1
    """
).get_as_pl()

id,title,description,variety,points,price,region_1
i64,str,str,str,i64,f64,str
0,"""Nicosia 2013 Vulkà Bianco (Et…","""Aromas include tropical fruit,…","""White Blend""",87,null,"""Etna"""
1,"""Quinta dos Avidagos 2011 Avida…","""This is ripe and fruity, a win…","""Portuguese Red""",87,15.0,null
2,"""Rainstorm 2013 Pinot Gris (Wil…","""Tart and snappy, the flavors o…","""Pinot Gris""",87,14.0,"""Willamette Valley"""
3,"""St. Julian 2013 Reserve Late H…","""Pineapple rind, lemon pith and…","""Riesling""",87,13.0,"""Lake Michigan Shore"""
4,"""Sweet Cheeks 2012 Vintner's Re…","""Much like the regular bottling…","""Pinot Noir""",87,65.0,"""Willamette Valley"""
…,…,…,…,…,…,…
129966,"""Dr. H. Thanisch (Erben Müller-…","""Notes of honeysuckle and canta…","""Riesling""",90,28.0,null
129967,"""Citation 2004 Pinot Noir (Oreg…","""Citation is given as much as a…","""Pinot Noir""",90,75.0,"""Oregon"""
129968,"""Domaine Gresser 2013 Kritt Gew…","""Well-drained gravel soil gives…","""Gewürztraminer""",90,30.0,"""Alsace"""


In [28]:
kuzu_conn.execute(
    """
    LOAD FROM 'final/winemag-reviews.parquet' RETURN
        taster_name,
        taster_twitter_handle
    """
).get_as_pl()

taster_name,taster_twitter_handle
str,str
"""Kerin O’Keefe""","""@kerinokeefe"""
"""Roger Voss""","""@vossroger"""
"""Paul Gregutt""","""@paulgwine """
"""Alexander Peartree""",null
"""Paul Gregutt""","""@paulgwine """
…,…
"""Anna Lee C. Iijima""",null
"""Paul Gregutt""","""@paulgwine """
"""Roger Voss""","""@vossroger"""


### Insert data into the graph
Using the direct scanning functionality from existing data formats, we can insert the data into
the node and rel tables. This is done by using the `COPY FROM` query in conjunction with the
`LOAD FROM` subquery.

In [29]:
# Insert wine data into Kùzu graph
kuzu_conn.execute(
    """
    COPY Wine FROM (
        LOAD FROM 'final/winemag-reviews.parquet' RETURN
            id,
            title,
            description,
            variety,
            points,
            price,
            region_1
        )
    """
)

In [30]:
# Insert Taster and Country data into Kùzu graph
kuzu_conn.execute(
    """
    COPY Taster FROM (
        LOAD FROM 'final/winemag-reviews.parquet'
            WHERE taster_name IS NOT NULL
            RETURN DISTINCT
                taster_name,
                taster_twitter_handle
        )
    """
)

kuzu_conn.execute(
    """
    COPY Country FROM (
        LOAD FROM 'final/winemag-reviews.parquet'
            WHERE country IS NOT NULL
            RETURN DISTINCT country
        )
    """
)

In [31]:
# Insert relationships by only selecting FROM and TO columns
kuzu_conn.execute(
    """
    COPY Tastes FROM (
        LOAD FROM 'final/winemag-reviews.parquet'
            WHERE taster_name IS NOT NULL
            RETURN
                taster_name,
                id
        )
    """
)

kuzu_conn.execute(
    """
    COPY IsFrom FROM (
        LOAD FROM 'final/winemag-reviews.parquet'
            WHERE country IS NOT NULL
            RETURN
                id,
                country
        )
    """
)

## Next steps
The data has been successfully loaded into Kùzu and can be queried using Cypher. The next steps would
be to inspect the graph visually using [Kùzu Explorer](https://docs.kuzudb.com/visualization/), and
to iterate on the data model.

We can run a simple summarization query in Cypher to see the number of wines tasted by each taster.

In [32]:
kuzu_conn.execute(
    """
    MATCH (t:Taster)-[:Tastes]->(w:Wine)
    RETURN t.taster_name AS taster, count(w) AS numWinesTasted
    ORDER BY count(w) DESC LIMIT 5
    """
).get_as_pl()

taster,numWinesTasted
str,i64
"""Roger Voss""",25514
"""Michael Schachner""",15134
"""Kerin O’Keefe""",10776
"""Virginie Boone""",9537
"""Paul Gregutt""",9532
